# Lab 12: Photometry
Based on the AAS2016 photuntls tutorial.
- Documentation: https://photutils.readthedocs.io/en/stable/


Name: 

## What is Aperture Photometry?
Aperture photometry is one method we use to convert astronomical images into measurements of brightness for individual stars. In this particular case we will use circular apertures for stars, but the same basic principals work for ellipse etc. around galaxies and other diffuse objects. The fundamental idea is to sum up the number of counts in the aperture to determine how many photons where observed. We also need to subtract off background photons.

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import scipy.stats

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['image.cmap'] = 'Greys_r'

# run the %matplotlib magic command to enable inline plotting
# in the current Notebook
%matplotlib inline

## Load our image

In [ ]:
#Read in our image
hdulist = fits.open('data/aa_aql0007.fits')
hdulist.info()
prihdr = hdulist[0].header
image_data = hdulist[0].data.astype(np.float) #Ensure a good data type
image_data = image_data[0:600,0:600] #Shrink our image

In [ ]:
#Get Useful header info
read_noise = np.float(prihdr['rdnoise'])
gain = np.float(prihdr['gain'])
exptime = np.float(prihdr['exptime'])
print(read_noise)
print(gain)
print(exptime)

## Gain
The image needs to be in electrons per second

In [ ]:
image_data = image_data * gain

In [ ]:
#View our image
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()

## Defining Apertures
For stellar photometry we will use circular apertures. Let's take an aperture radius 15 pixels around a star at (439.795,502.256) python space. Apertures are defined using (x,y) instead of (y,x) and start counting from (0,0). Apertures can be defined in several different ways because we are going from a continuous space to a discrete one. 
![](data/photutils_aperture_methods.svg)

In [ ]:
from photutils import CircularAperture, aperture_photometry
# define the aperture
position = (502.256,439.795)
radius = 15
aperture = CircularAperture(position, r=radius)

In [ ]:
# center method
phot = aperture_photometry(image_data, aperture,  method='center')
phot

In [ ]:
# subpixel method, subpixels=5 (same as Source Extractor (SExtractor))
phot = aperture_photometry(image_data, aperture,  method='subpixel', subpixels=5)
phot

In [ ]:
# perform the photometry; the default method is 'exact'
phot = aperture_photometry(image_data, aperture)
phot

## Photometric Error
The Error in our photometry comes from two sources counting/Poisson noise and readnoise. We will assume that these two noise sources are independent, so we will add them in quadrature. Poisson noise is the square root of the counts.

In [ ]:
fixed_data = image_data
fixed_data[(fixed_data < 0)] = 0
poisson = np.sqrt(fixed_data/gain)
error = np.sqrt(poisson**2 + read_noise**2)

In [ ]:
# input the data units
import astropy.units as u
unit = u.electron / u.s
phot = aperture_photometry(image_data, aperture, error=error, unit=unit)
phot

## Multiple Positions

In [ ]:
positions = [(502.256,439.795), (354.291,363.823)]
radius = 15.
apertures = CircularAperture(positions, r=radius)
phot = aperture_photometry(image_data, apertures, error=error,unit=unit)
phot

In [ ]:
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()
apertures.plot(color='blue')

## Measure Sky Background
We want to remove the sky background from our pixels. Note the sky background does contribute to the noise

In [ ]:
from photutils import CircularAnnulus
bkg_apertures = CircularAnnulus(positions, r_in=30., r_out=35.)

In [ ]:
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()
apertures.plot(color='blue')
bkg_apertures.plot(color='cyan', hatch='//', alpha=0.8)

In [ ]:
# measure the aperture sum for the star
phot = aperture_photometry(image_data, apertures,error=error)

In [ ]:
#It turns out that the best measurement of the backgroud for photometry is not the mean, but rather the mode
def getBackground(image_data, apertures):
    bkgs = list()
    for m in apertures.to_mask('center'):
        inc_pix = m.cutout(image_data)[m.cutout(image_data) *m.data >0]
        mode_bak = scipy.stats.mode(inc_pix)
        bkgs.append(float(mode_bak[0]))
    return np.array(bkgs)

In [ ]:
#Get the mode of the background for each stars
bkg_mode = getBackground(image_data,bkg_apertures)
print(bkg_mode)

In [ ]:
# now calculate the total background in the circular aperture
bkg_sum = bkg_mode * apertures.area()

phot['bkg_sum'] = bkg_sum
phot

In [ ]:
# subtract the background
flux_bkgsub = phot['aperture_sum'] - bkg_sum

phot['aperture_sum_bkgsub'] = flux_bkgsub
phot

## Convert to Magnitudes
The final step is to convert to magnitudes. Note I am being very loose with some of the errors. We are only including Poisson error at the moment.

In [ ]:
zpt = 25.
mag = zpt - 2.5*np.log10(phot['aperture_sum_bkgsub']) + 2.5*np.log10(exptime)
merr = 1.0857 *phot['aperture_sum_err']/phot['aperture_sum_bkgsub']
phot['Mag'] = mag
phot['Mag_err'] = merr
phot

## Now it is your turn.
Find the magnitude for the three bright stars in the lower left corner of the image.